In [1]:
from html.parser import HTMLParser


def attr_value(attrs, attr):
    for (name, value) in attrs:
        if name == attr:
            return value.split(' ')
    return []

class AgendaParser(HTMLParser):
    values = []

    def handle_starttag(self, tag, attrs):
        if tag == 'div' and 'event' in attr_value(attrs, 'class'):
            event_kind = 'teams' if 'Teams' in attr_value(attrs, 'class') else 'other'
            self.values.append([event_kind, 'href', 'startDate'])

        if tag == 'a' and 'evenement-tegel-link' in attr_value(attrs, 'class'):
            href = attr_value(attrs, 'href')[0]
            self.values[-1][1] = href

        if tag == 'time' and 'startDate' in attr_value(attrs, 'itemprop'):
            start_date = attr_value(attrs, 'datetime')[0]
            self.values[-1][2] = start_date

In [ ]:
import requests

base_url = "https://www.nklegpuzzelen.nl/"

agenda_parser = AgendaParser()

r = requests.get(base_url + 'agenda-inschrijven/')
agenda_parser.feed(r.text)

In [3]:
from_date = '2024-06-01'
till_date = '2025-03-01'

preliminary_events = []
final_event = None

for line in agenda_parser.values:
    if line[0] == 'teams' and from_date < line[2] and line[2] < till_date:
        preliminary_events.append(line)


for line in agenda_parser.values:
    if line[0] == 'teams' and line[2] > till_date:
        final_event = line
        final_event[0] = 'final'

In [5]:
class EventParser(HTMLParser):
    track_data = False
    td_open = False
    values = []

    def handle_starttag(self, tag, attrs):
        if tag == 'div' and 'mod_mod_nk_livestanden' in attr_value(attrs, 'class'):
            self.track_data = True
        
        if tag == 'td':
            self.td_open = True
    
    def handle_endtag(self, tag):
        if tag == 'div':
            self.track_data = False

        if tag == 'td':
            self.td_open = False
        
    def handle_data(self, data):
        if self.track_data and self.td_open:
            value = data.strip()
            if len(self.values) == 0 or len(self.values[-1]) == 3:
                self.values.append([value])
            else:
                self.values[-1].append(value)

    def reset2(self):
        self.track_data = False
        self.td_open = False
        self.values = []

In [7]:
import csv
import requests
import os

os.makedirs('data', exist_ok=True)

base_url = "https://www.nklegpuzzelen.nl/"
event_parser = EventParser()

for line in preliminary_events + [final_event]:
    title = 'final' if line[0] == 'final' else line[1].split('/')[-1]
    print(title)

    r = requests.get(base_url + line[1])
    
    event_parser.feed(r.text)

    with open(f'data/{title}.csv', 'w') as f:
        writer = csv.writer(f)
        for line in event_parser.values:
            writer.writerow(line)
    
    event_parser.reset2()

teteringen
obdam-2
obdam
roelofarendsveen-4
roelofarendsveen-3
roelofarendsveen-2
zwartebroek
meterik
reek-2
sleeuwijk
vorden
hooghalen
de-lier-2
de-lier
meppel
final
